In [9]:
experiments = [
    {
        "dataset": {
            "video_path": r"H:\My Drive\Megatron_DeepFake\dataset",
            "num_frames": 5,
            "random_initial_frame": False,
            "depth_anything_size": "Small",
            "num_video" : 100
        },
        "dataloader": {
            "batch_size": 8,
            "repvit_model": "repvit_m0_9.dist_300e_in1k",
            "num_workers": 0,
            "pin_memory":False
        },
        "transformer": {
            "d_model": 384,
            "n_heads": 2,
            "n_layers": 1,
            "d_ff": 1024,
        },
        "train": {
            "learning_rate": 0.001,
            "epochs": 3,
            "tmp_dir" : "./../tmp",
            "log_dir": "./../data/runs/exp1",
            "early_stop_counter": 10,
            "resume_training": False,
            "train_size": 0.5,
            "val_size": 0.3,
            "test_size": 0.2,
        },
        "seed": 42,
    }
]

In [10]:
import torch 
import random
import numpy as np
from megatron.trainer import Trainer
from megatron.configuration import ExperimentConfig
import time

# Set cuda operations deterministic
torch.backends.cudnn.deterministic = True

config = ExperimentConfig(**experiments[0])
random.seed(config.seed)
np.random.seed(config.seed)
torch.manual_seed(config.seed)
trainer = Trainer(config)



start_time = time.time() 
trainer.optimized_train_and_validate()
end_time = time.time()

# Calculate the duration in seconds
duration_seconds = end_time - start_time
hours, remainder = divmod(duration_seconds, 3600)
minutes, seconds = divmod(remainder, 60)
milliseconds = int((seconds - int(seconds)) * 1000)
seconds = int(seconds)

print(f"Optimized train with 3 epochs and validate without cache: {hours}h:{minutes}m:{seconds}s:{milliseconds}ms")

# Try to dealloc
trainer = None
del trainer

Caching train data:   0%|          | 0/63 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import torch 
import random
import numpy as np
from megatron.trainer import Trainer
from megatron.configuration import ExperimentConfig
import time

# Set cuda operations deterministic
torch.backends.cudnn.deterministic = True

config = ExperimentConfig(**experiments[0])
random.seed(config.seed)
np.random.seed(config.seed)
torch.manual_seed(config.seed)
config.train.epochs = 100
trainer = Trainer(config)

start_time = time.time() 
trainer.optimized_train_and_validate()
end_time = time.time()

# Calculate the duration in seconds
duration_seconds = end_time - start_time
hours, remainder = divmod(duration_seconds, 3600)
minutes, seconds = divmod(remainder, 60)
milliseconds = int((seconds - int(seconds)) * 1000)
seconds = int(seconds)

print(f"Optimized train  and validate with cache, 100 epochs: {hours}h:{minutes}m:{seconds}s:{milliseconds}ms")

# Try to dealloc
trainer = None
del trainer

In [ ]:
import torch 
import random
import numpy as np
from megatron.trainer import Trainer
from megatron.configuration import ExperimentConfig
import time

# Set cuda operations deterministic
torch.backends.cudnn.deterministic = True

config = ExperimentConfig(**experiments[0])
random.seed(config.seed)
np.random.seed(config.seed)
torch.manual_seed(config.seed)
trainer = Trainer(config)

start_time = time.time() 
trainer.train_and_validate()
end_time = time.time()

# Calculate the duration in seconds
duration_seconds = end_time - start_time
hours, remainder = divmod(duration_seconds, 3600)
minutes, seconds = divmod(remainder, 60)
milliseconds = int((seconds - int(seconds)) * 1000)
seconds = int(seconds)

print(f"Unptimized train and validate: {hours}h:{minutes}m:{seconds}s:{milliseconds}ms")

# Try to dealloc
trainer = None
del trainer

In [ ]:
from megatron.model import Megatron
from pathlib import Path

config = ExperimentConfig(**experiments[0])
model = Megatron(
    depth_anything_size = config.dataset.depth_anything_size,
    repvit_model= config.dataloader.repvit_model,
    d_model = config.transformer.d_model,
    max_len_pe=config.dataset.num_video,
    n_heads = config.transformer.n_heads,
    n_layers = config.transformer.n_layers,
    d_ff  = config.transformer.d_ff,
).from_pretrained(Path(config.train.log_dir))
model.inference(video_path =Path(r"H:\My Drive\Megatron_DeepFake\dataset\manipulated_sequences\DeepFakeDetection\raw\videos\01_03__hugging_happy__ISF9SP4G.mp4"))